In [9]:
import pandas as pd
import os

In [130]:
results_total = pd.DataFrame()
for file in os.listdir():
    if pd.Series(file).str.contains('Results')[0]:
        results = pd.read_csv(file)
        results['file'] = str(file)
        results_total = pd.concat([results_total,results])

model = results_total['Unnamed: 0'].str.split('-',expand = True)
model.columns = ['Variable', 'Model_Info', 'Threshold', 'Execution Time (s)']
model['Execution Time (s)'] = model['Execution Time (s)'].str.split(':',expand=True)[1]
model['Threshold'] = model['Threshold'].str.split(' * ',expand=True)[4]
model['Model'] = model['Model_Info'].str.split('(',expand=True)[0]
model['Contamination'] = model['Model_Info'].str.split('(',expand=True)[1].str.split('contamination=',expand=True)[1].str.split(',',expand=True)[0]
model['n_neighbors'] = model['Model_Info'].str.split('(',expand=True)[1].str.split('n_neighbors=',expand=True)[1].str.split(',',expand=True)[0].str.split(')',expand=True)[0]
model['n_estimators'] = model['Model_Info'].str.split('(',expand=True)[1].str.split('n_estimators=',expand=True)[1].str.split(',',expand=True)[0]

results_total = pd.concat([results_total,model], axis=1)

results_total['Model Type'] = results_total.file.str.split('_',expand=True)[0].str.split('Results',expand=True)[0]

results_total['Training %'] = results_total.file.str.split('_',expand=True)[1].str.split('.',expand=True)[0]

results_total = results_total[['Variable','Model', 'Model_Info', 'Model Type', 'Training %','Precision', 'Recall', 'Accuracy', 'F1-score', 'AUC',
       'Train Ratio', 'Number of true anomalies',
       'Number of predicted anomalies', 'True Positives', 'False Positives',
       'True Negatives', 'False Negatives',
       'Threshold', 'Execution Time (s)', 'Contamination',
       'n_neighbors', 'n_estimators']]

In [153]:
results_total.Contamination.value_counts()

Contamination
0.005    120
0.01     120
0.05     120
0.1      120
0.15     120
0.001    108
0.091     12
Name: count, dtype: int64

In [170]:
df = df[df['Variable'] == 'AC_Power_B (KW) ']

,Variable,Model,Model_Info,Model Type,Training %,Precision,Recall,Accuracy,F1-score,AUC,...,Number of predicted anomalies,True Positives,False Positives,True Negatives,False Negatives,Threshold,Execution Time (s),Contamination,n_neighbors,n_estimators
60,AC_Power_B (KW),LOF,"LOF(algorithm='auto', contamination=0.001, le...",SemiSupervised,70,0.52,0.41,1.00,0.46,0.70,...,46,24,22,78486,35,0.0,0.366302490234375,0.001,10,None
61,AC_Power_B (KW),LOF,"LOF(algorithm='auto', contamination=0.005, le...",SemiSupervised,70,0.20,0.78,1.00,0.32,0.89,...,226,46,180,78328,13,0.0,0.39212536811828613,0.005,10,None
62,AC_Power_B (KW),LOF,"LOF(algorithm='auto', contamination=0.01, lea...",SemiSupervised,70,0.09,0.78,0.99,0.17,0.89,...,487,46,441,78067,13,0.0,0.3663063049316406,0.01,10,None
63,AC_Power_B (KW),LOF,"LOF(algorithm='auto', contamination=0.05, lea...",SemiSupervised,70,0.01,0.78,0.96,0.03,0.87,...,3427,46,3381,75127,13,0.0,0.35805296897888184,0.05,10,None
64,AC_Power_B (KW),LOF,"LOF(algorithm='auto', contamination=0.1, leaf...",SemiSupervised,70,0.01,0.78,0.90,0.01,0.84,...,7786,46,7740,70768,13,0.0,0.3625450134277344,0.1,10,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,AC_Power_B (KW),ECOD,"ECOD(contamination=0.005, n_jobs=1)",Unsupervised,90,0.00,0.57,0.91,0.00,0.74,...,2355,4,2351,23831,3,7.0,0.1306140422821045,0.005,None,None
116,AC_Power_B (KW),ECOD,"ECOD(contamination=0.01, n_jobs=1)",Unsupervised,90,0.00,0.57,0.91,0.00,0.74,...,2355,4,2351,23831,3,6.21,0.14786458015441895,0.01,None,None
117,AC_Power_B (KW),ECOD,"ECOD(contamination=0.05, n_jobs=1)",Unsupervised,90,0.00,0.57,0.91,0.00,0.74,...,2355,4,2351,23831,3,3.87,0.12284278869628906,0.05,None,None
118,AC_Power_B (KW),ECOD,"ECOD(contamination=0.1, n_jobs=1)",Unsupervised,90,0.00,0.57,0.91,0.00,0.74,...,2355,4,2351,23831,3,2.89,0.12979459762573242,0.1,None,None


In [180]:
df = df.reset_index(drop=True)

In [181]:
df.loc[df.groupby(['Variable','Model', 'Model Type','Training %','Contamination']).AUC.idxmax().values,:]

,Variable,Model,Model_Info,Model Type,Training %,Precision,Recall,Accuracy,F1-score,AUC,...,Number of predicted anomalies,True Positives,False Positives,True Negatives,False Negatives,Threshold,Execution Time (s),Contamination,n_neighbors,n_estimators
42,AC_Power_B (KW),ABOD,"ABOD(contamination=0.001, method='fast', n_ne...",SemiSupervised,70,0.00,0.00,1.00,0.00,0.50,...,0,0,0,78508,59,nan,7.328582525253296,0.001,5,None
43,AC_Power_B (KW),ABOD,"ABOD(contamination=0.005, method='fast', n_ne...",SemiSupervised,70,0.00,0.00,1.00,0.00,0.50,...,0,0,0,78508,59,nan,7.417108058929443,0.005,5,None
44,AC_Power_B (KW),ABOD,"ABOD(contamination=0.01, method='fast', n_nei...",SemiSupervised,70,0.00,0.00,1.00,0.00,0.50,...,0,0,0,78508,59,nan,7.481013298034668,0.01,5,None
45,AC_Power_B (KW),ABOD,"ABOD(contamination=0.05, method='fast', n_nei...",SemiSupervised,70,0.00,0.00,1.00,0.00,0.50,...,0,0,0,78508,59,nan,7.456040382385254,0.05,5,None
46,AC_Power_B (KW),ABOD,"ABOD(contamination=0.1, method='fast', n_neig...",SemiSupervised,70,0.00,0.00,1.00,0.00,0.50,...,0,0,0,78508,59,nan,7.41864800453186,0.1,5,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,AC_Power_B (KW),LOF,"LOF(algorithm='auto', contamination=0.01, lea...",Unsupervised,90,0.02,0.57,0.99,0.03,0.78,...,266,4,262,25920,3,0.0,7.57984733581543,0.01,30,None
309,AC_Power_B (KW),LOF,"LOF(algorithm='auto', contamination=0.05, lea...",Unsupervised,90,0.00,0.57,0.96,0.01,0.76,...,1104,4,1100,25082,3,0.0,7.546696186065674,0.05,30,None
312,AC_Power_B (KW),LOF,"LOF(algorithm='auto', contamination=0.091, le...",Unsupervised,90,0.00,0.57,0.94,0.00,0.75,...,1638,4,1634,24548,3,0.0,8.074262619018555,0.091,50,None
310,AC_Power_B (KW),LOF,"LOF(algorithm='auto', contamination=0.1, leaf...",Unsupervised,90,0.00,0.57,0.96,0.01,0.76,...,1104,4,1100,25082,3,0.0,7.56572151184082,0.1,30,None


In [179]:
df

,Variable,Model,Model_Info,Model Type,Training %,Precision,Recall,Accuracy,F1-score,AUC,...,Number of predicted anomalies,True Positives,False Positives,True Negatives,False Negatives,Threshold,Execution Time (s),Contamination,n_neighbors,n_estimators
60,AC_Power_B (KW),LOF,"LOF(algorithm='auto', contamination=0.001, le...",SemiSupervised,70,0.52,0.41,1.00,0.46,0.70,...,46,24,22,78486,35,0.0,0.366302490234375,0.001,10,None
61,AC_Power_B (KW),LOF,"LOF(algorithm='auto', contamination=0.005, le...",SemiSupervised,70,0.20,0.78,1.00,0.32,0.89,...,226,46,180,78328,13,0.0,0.39212536811828613,0.005,10,None
62,AC_Power_B (KW),LOF,"LOF(algorithm='auto', contamination=0.01, lea...",SemiSupervised,70,0.09,0.78,0.99,0.17,0.89,...,487,46,441,78067,13,0.0,0.3663063049316406,0.01,10,None
63,AC_Power_B (KW),LOF,"LOF(algorithm='auto', contamination=0.05, lea...",SemiSupervised,70,0.01,0.78,0.96,0.03,0.87,...,3427,46,3381,75127,13,0.0,0.35805296897888184,0.05,10,None
64,AC_Power_B (KW),LOF,"LOF(algorithm='auto', contamination=0.1, leaf...",SemiSupervised,70,0.01,0.78,0.90,0.01,0.84,...,7786,46,7740,70768,13,0.0,0.3625450134277344,0.1,10,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,AC_Power_B (KW),ECOD,"ECOD(contamination=0.005, n_jobs=1)",Unsupervised,90,0.00,0.57,0.91,0.00,0.74,...,2355,4,2351,23831,3,7.0,0.1306140422821045,0.005,None,None
116,AC_Power_B (KW),ECOD,"ECOD(contamination=0.01, n_jobs=1)",Unsupervised,90,0.00,0.57,0.91,0.00,0.74,...,2355,4,2351,23831,3,6.21,0.14786458015441895,0.01,None,None
117,AC_Power_B (KW),ECOD,"ECOD(contamination=0.05, n_jobs=1)",Unsupervised,90,0.00,0.57,0.91,0.00,0.74,...,2355,4,2351,23831,3,3.87,0.12284278869628906,0.05,None,None
118,AC_Power_B (KW),ECOD,"ECOD(contamination=0.1, n_jobs=1)",Unsupervised,90,0.00,0.57,0.91,0.00,0.74,...,2355,4,2351,23831,3,2.89,0.12979459762573242,0.1,None,None


In [186]:
import plotly.express as px
df = results_total.copy()
df = df.reset_index(drop=True)
df = df[df['Variable'] == 'AC_Power_B (KW) ']
df = df.loc[df.groupby(['Variable','Model', 'Model Type','Training %','Contamination']).AUC.idxmax().values,:].sort_values(by='AUC', ascending = False)
fig = px.bar(df, x="Model", y="AUC", color="Model Type", barmode="group",facet_row="Contamination", facet_col="Training %",width=2000, height=2000)
fig.show()

In [188]:
import plotly.express as px
df = results_total.copy()
df = df.reset_index(drop=True)
df = df[df['Variable'] == 'AC_Power_B (KW) ']
df = df.loc[df.groupby(['Variable','Model', 'Model Type','Training %','Contamination']).AUC.idxmax().values,:].sort_values(by='F1-score', ascending = False)
fig = px.bar(df, x="Model", y="F1-score", color="Model Type", barmode="group",facet_row="Contamination", facet_col="Training %",width=2000, height=2000)
fig.show()

In [187]:
df

,Variable,Model,Model_Info,Model Type,Training %,Precision,Recall,Accuracy,F1-score,AUC,...,Number of predicted anomalies,True Positives,False Positives,True Negatives,False Negatives,Threshold,Execution Time (s),Contamination,n_neighbors,n_estimators
97,AC_Power_B (KW),KNN,"KNN(algorithm='auto', contamination=0.005, le...",SemiSupervised,70,0.55,0.78,1.00,0.64,0.89,...,84,46,38,78470,13,0.0,6.758800745010376,0.005,5,None
98,AC_Power_B (KW),KNN,"KNN(algorithm='auto', contamination=0.01, lea...",SemiSupervised,70,0.45,0.78,1.00,0.57,0.89,...,103,46,57,78451,13,0.0,6.807952880859375,0.01,5,None
62,AC_Power_B (KW),LOF,"LOF(algorithm='auto', contamination=0.01, lea...",SemiSupervised,70,0.09,0.78,0.99,0.17,0.89,...,487,46,441,78067,13,0.0,0.3663063049316406,0.01,10,None
456,AC_Power_B (KW),KNN,"KNN(algorithm='auto', contamination=0.001, le...",Unsupervised,70,0.67,0.78,1.00,0.72,0.89,...,69,46,23,78485,13,0.01,12.859455347061157,0.001,5,None
61,AC_Power_B (KW),LOF,"LOF(algorithm='auto', contamination=0.005, le...",SemiSupervised,70,0.20,0.78,1.00,0.32,0.89,...,226,46,180,78328,13,0.0,0.39212536811828613,0.005,10,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467,AC_Power_B (KW),ABOD,"ABOD(contamination=0.15, method='fast', n_nei...",Unsupervised,70,0.00,0.00,1.00,0.00,0.50,...,0,0,0,78508,59,nan,18.243993043899536,0.15,5,None
466,AC_Power_B (KW),ABOD,"ABOD(contamination=0.1, method='fast', n_neig...",Unsupervised,70,0.00,0.00,1.00,0.00,0.50,...,0,0,0,78508,59,nan,17.92073106765747,0.1,5,None
465,AC_Power_B (KW),ABOD,"ABOD(contamination=0.05, method='fast', n_nei...",Unsupervised,70,0.00,0.00,1.00,0.00,0.50,...,0,0,0,78508,59,nan,18.15277862548828,0.05,5,None
464,AC_Power_B (KW),ABOD,"ABOD(contamination=0.01, method='fast', n_nei...",Unsupervised,70,0.00,0.00,1.00,0.00,0.50,...,0,0,0,78508,59,nan,18.188961505889893,0.01,5,None
